In [1]:
from models.gwn import gwnet
from utils.data_loader import load_data_PEMS_BAY
from data.preprocess_pemsbay import generate_dataset, get_normalized_adj
import argparse
import os
import numpy as np
import torch
import torch.nn as nn


In [2]:
num_timesteps_input = 12
num_timesteps_output = 4
epochs = 100
batch_size = 64
input = "D:\Research\PEMSBAY"
device = torch.device('cuda')

In [3]:
A, X, means, stds = load_data_PEMS_BAY(input) # (N, F, T)
split_line = int(X.shape[2] * 0.1)

train_original_data = X[:, :, :split_line]
test_original_data = X[:, :, split_line:]

training_input, training_target = generate_dataset(train_original_data,
                                                    num_timesteps_input=num_timesteps_input,
                                                    num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                            num_timesteps_input=num_timesteps_input,
                                            num_timesteps_output=num_timesteps_output)

# (B, N, T, F), (B, N, T, F)

A_wave = get_normalized_adj(A)
A_wave = torch.from_numpy(A_wave).float()
A_wave = A_wave.to(device=device)


In [4]:
print(training_input.shape)
print(A_wave.shape)

torch.Size([2605, 863, 12, 3])
torch.Size([863, 863])


In [5]:
supports = [A_wave]
aptinit = supports[0]

In [6]:
for i in supports:
    print(i.shape)

torch.Size([863, 863])


In [7]:
print(aptinit.shape)

torch.Size([863, 863])


In [8]:
permutation = torch.randperm(training_input.shape[0])
indices = permutation[0:0 + 1]
X_batch, y_batch = training_input[indices], training_target[indices]

In [13]:
input = X_batch.to(device)

In [10]:
print(input.shape)

torch.Size([1, 863, 12, 3])


In [14]:
model = gwnet(device, num_nodes=A_wave.shape[0], in_feature=3, out_feature=3, num_step_output=4, supports=supports, aptinit=aptinit).to(device)

In [15]:
output = model(A_wave, input)

Input shape vào model: torch.Size([1, 3, 863, 13])
1:  torch.Size([1, 3, 863, 13])
2:  torch.Size([1, 32, 863, 13])
3:  torch.Size([1, 32, 863, 1])
4:  torch.Size([1, 256, 863, 1])
5:  torch.Size([1, 512, 863, 1])
6:  torch.Size([1, 12, 863, 1])
7:  torch.Size([1, 863, 12, 1])


In [16]:
model2 = gwnet(device, num_nodes=A_wave.shape[0], in_feature=3, out_feature=3, num_step_output=4).to(device)
output2 = model2(A_wave, input)

Input shape vào model: torch.Size([1, 3, 863, 13])
1:  torch.Size([1, 3, 863, 13])
2:  torch.Size([1, 32, 863, 13])
3:  torch.Size([1, 32, 863, 1])
4:  torch.Size([1, 256, 863, 1])
5:  torch.Size([1, 512, 863, 1])
6:  torch.Size([1, 12, 863, 1])
7:  torch.Size([1, 863, 12, 1])


In [17]:
output.shape

torch.Size([1, 863, 4, 3])

In [18]:
torch.cuda.empty_cache()